<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/teste_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pacotes
import os
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy import stats
import pickle
from PIL import Image, ImageDraw, ImageFont
#
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.backend import expand_dims, repeat_elements
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import ConvLSTM2D, Bidirectional, Input, Permute, Reshape, TimeDistributed, Lambda, Conv2D, Dense, Dropout, Normalization

In [ ]:
from google.colab import drive
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [ ]:
# Não precisa rodar de novo
# Checando
# path_checar = "/content/drive/My Drive/projeto_DL/dataset_tratado/patches/"
# print(len(os.listdir(path_checar))==6000) # Deveria ter 6000

In [ ]:
# DataGenerator de https://github.com/mauriciodev/tec_forecast/blob/main/examples/tec_forecast_tf2_lab.ipynb
# split_sequence alterada para levar em conta que estamos trabalhando com patches na linha 71
# e também o self.list_IDs da linha 9
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, x, batch_size, nstepsin=4, nstepsout=1, shuffle=True,training=True, sample_rate=12, val_split=0, validation=False, random_state=23):
        'Initialization'
        super().__init__()
        self.list_IDs=range(0,len(x)-(nstepsout-1+nstepsin)-100,sample_rate) #store the index to allow shuffling
        if val_split>0:
            datasplit=train_test_split(self.list_IDs,random_state=random_state, test_size=val_split)
            if validation==False: #training generator
                self.list_IDs=datasplit[0]
            else:
                self.list_IDs=datasplit[1]
        self.x=x
        self.nstepsin=nstepsin
        self.nstepsout=nstepsout
        self.batch_size=batch_size
        self.dim=x[0].shape
        self.shuffle=shuffle
        self.training=training
        self.on_epoch_end()

    def count(self):
        'Returns the number of samples'
        return len(self.list_IDs)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        if self.training:
            return X, y
        else:
            return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        Y = []
        # Generate data
        for ID in list_IDs_temp:
            x,y=self.split_sequence(ID)
            X.append(x)
            Y.append(y) # <- alteração aqui
        return np.array(X), np.array(Y)

    def split_sequence(self, i):
        # find the end of this pattern
        end_ix = i + self.nstepsin
        # check if we are beyond the sequence
        if end_ix + self.nstepsout> len(self.x):
            return None,None
        # gather input and output parts of the pattern
        seq_x, seq_y = self.x[i:end_ix], self.x[(end_ix+100):(end_ix+self.nstepsout+100)]
        return seq_x,seq_y

    def asArray(self):
        return self.__data_generation(self.list_IDs)

In [ ]:
# Analisando as imagens parece que as primeiras 12 não apresentam variação.
# Acho que vale a pena descartar.
# Vou fazer isso
path = "/content/drive/My Drive/projeto_DL/dataset_tratado/patches/"
arqs = []
for i in ['2017','2018','2019','2020','2021']:
  for j in range(12):
    for k in range(100):
      arqs.append(path+i+"_"+str(j+1)+"_patch_"+str(k+1)+".pkl")

print(arqs)
print(len(arqs))

['/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_1.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_2.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_3.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_4.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_5.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_6.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_7.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_8.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_9.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_10.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_11.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/2017_1_patch_12.pkl', '/content/drive/My Drive

In [ ]:
# Não precisa rodar de novo
# Separando dados
# train_npy = []
# test_npy = []

# contador = len(arqs)
# for i in range(len(arqs)):
#   with open(arqs[i], 'rb') as handle:
#     dados = pickle.load(handle)
#     dados = dados.astype('float32')
#     dados = np.expand_dims(dados, axis=2)
#
#   if contador <= 1200:
#     test_npy.append(dados)
#   else:
#     train_npy.append(dados)
#
#   if ((6000-contador) % 50) == 0:
#           print("contador = " + str(6000-contador))
#   contador = contador-1
#   time.sleep(0.05)

# Salvando bases acima
# with open("/content/drive/My Drive/projeto_DL/dataset_tratado/train_npy.pkl", 'wb') as handle:
#           pickle.dump(train_npy, handle, protocol=pickle.HIGHEST_PROTOCOL)
#           time.sleep(0.5)

# with open("/content/drive/My Drive/projeto_DL/dataset_tratado/test_npy.pkl", 'wb') as handle:
#           pickle.dump(test_npy, handle, protocol=pickle.HIGHEST_PROTOCOL)
#           time.sleep(0.5)


In [ ]:
# Abrindo dados
with open("/content/drive/My Drive/projeto_DL/dataset_tratado/train_npy.pkl", 'rb') as handle:
  train_npy = pickle.load(handle)
  time.sleep(0.5)

with open("/content/drive/My Drive/projeto_DL/dataset_tratado/test_npy.pkl", 'rb') as handle:
  test_npy = pickle.load(handle)
  time.sleep(0.5)

print(len(train_npy))
print(len(test_npy))

4800
1200


In [ ]:
# Alterei a saída para sigmoide
def ed_convlstm(inputShape,filters=16,nstepsout=12, kernel=(1, 1)):
    #Inspired on https://github.com/Azure/DeepLearningForTimeSeriesForecasting/blob/master/3_RNN_encoder_decoder.ipynb
    in_im = Input(shape=inputShape)
    x=in_im
    # Normalização
    x=Normalization()(x)
    #encoder
    x,h1,c1=ConvLSTM2D(filters=filters, kernel_size=kernel,padding='same',return_sequences=True,return_state=True)(x)
    x,h2,c2=ConvLSTM2D(filters=filters, kernel_size=kernel,padding='same',return_sequences=True,return_state=True)(x)
    x,h3,c3=ConvLSTM2D(filters=filters, kernel_size=kernel,padding='same',return_sequences=False,return_state=True)(x)
    x=Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), nstepsout, 1))(x)
    x=ConvLSTM2D(filters=filters, kernel_size=kernel,padding='same',return_sequences=True)(x, initial_state=[h1,c1])
    x=ConvLSTM2D(filters=filters, kernel_size=kernel,padding='same',return_sequences=True)(x, initial_state=[h2,c2])
    x=ConvLSTM2D(filters=filters, kernel_size=kernel,padding='same',return_sequences=True)(x, initial_state=[h3,c3])
    # x=TimeDistributed(Conv2D(1, (1, 1), activation='linear', padding='same'))(x) #changed to
    x=TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid', padding='same'))(x) #changed to
    return Model(in_im, x, name=sys._getframe().f_code.co_name)

In [ ]:
nstepsin=4 #5
nstepsout=1
time_stride=1
batch_size=16
random_seed=43
sample_rate_in=1
sample_rate_val_out=1

In [ ]:
train_set=DataGenerator(train_npy, batch_size=batch_size, nstepsin=nstepsin, nstepsout=nstepsout, shuffle=True, val_split=0.2,
                        validation=False, random_state=random_seed, sample_rate=sample_rate_in)
val_set=DataGenerator(train_npy, batch_size=batch_size, nstepsin=nstepsin, nstepsout=nstepsout, shuffle=True, val_split=0.2,
                      validation=True, random_state=random_seed, sample_rate=sample_rate_val_out)
test_set=DataGenerator(test_npy, batch_size=batch_size, nstepsin=nstepsin, nstepsout=nstepsout, shuffle=False, val_split=0,
                       validation=False, random_state=random_seed, sample_rate=sample_rate_val_out)

In [ ]:
print("X batches:",train_set[0][0].shape)
print("Y batches:",train_set[0][1].shape)
print("Training sSamples: ", train_set.count())

X batches: (16, 4, 256, 256, 1)
Y batches: (16, 1, 256, 256, 1)
Training sSamples:  3756


In [ ]:
inputShape=train_set[0][0][0].shape
model= ed_convlstm(inputShape, filters=12,kernel=(9,9), nstepsout=nstepsout)
# print(model.name)
# plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
# model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss=tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
earlystopping = tf.keras.callbacks.EarlyStopping(monitor= "val_accuracy", min_delta=0.0001,
                                                 patience=10, restore_best_weights=True,
                                                 verbose = 1, mode="max")
start = pd.Timestamp.now()
history=model.fit(train_set, validation_data=val_set, epochs=100, batch_size=batch_size,
                  callbacks = [earlystopping])
end = pd.Timestamp.now()
model.save("/content/drive/My Drive/projeto_DL/"+model.name+".h5")

Epoch 1/100
235/235 [==============================] - 522s 2s/step - loss: 0.5986 - accuracy: 0.6962 - val_loss: 0.5770 - val_accuracy: 0.6994
Epoch 2/100
235/235 [==============================] - 504s 2s/step - loss: 0.5815 - accuracy: 0.6965 - val_loss: 0.5696 - val_accuracy: 0.6994
Epoch 3/100
  2/235 [..............................] - ETA: 7:35 - loss: 0.5439 - accuracy: 0.7277

KeyboardInterrupt: ignored